# Outline of Part 2, Analysis of sorted spikes 

## 1) Set up, paths
## 2) Functions - easier than you think!
## 3) Load and inspect spikes
### a) Spike time raster 
### b) Binned raster
==========================================================================
### c) Zero in on a feature of data
## 4) Peristimulus time histogram

In [ ]:
# @title Helper functions
!pip install matplotlib pandas xarray numpy tqdm spykes

import os

import pandas as pd
import xarray as xr
import requests
from pathlib import Path
import zipfile


def download_data():
    # Get link:
    r = requests.get('http://data.cortexlab.net/singlePhase3/data/dataset.zip')
    path = Path('data/dataset.zip')
    path.parent.mkdir(exist_ok=True)
    if not path.exists():
        with open(path, "wb") as fid:
            # Write out content of link:
            fid.write(r.content)
    # Unzip
    with zipfile.ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall(path.parent)


def load_spikes_from_phy(path_to_data='/Users/myroshnychenkm2/Downloads/dataset/', sampling_frequency=30000):
    """
    Get spikes from a kilosort/phy result folder
    :param path_to_data:
    :param sampling_frequency:
    :return:
    :id: neuron id, 1xN
    :ts: corresponding spiketime, 1xN
    """
    groupfname = os.path.join(path_to_data, 'cluster_groups.csv')
    groups = pd.read_csv(groupfname, delimiter='\t')

    # load spike times and cluster IDs
    with open(path_to_data + 'spike_clusters.npy', 'rb') as f:
        ids = np.load(f).flatten()
    with open(path_to_data + 'spike_times.npy', 'rb') as f:
        ts = np.load(f).flatten()

    # Create the list of our "good" labeled units
    ids_to_take = groups[(groups.group == 'good')].cluster_id
    # Find which spikes beloing to our "good" groups
    spikes_to_take = []
    for i in tqdm(ids_to_take, desc='Selecting only good spikes'):
        spikes_to_take.extend((ids == i).nonzero()[0])
    # only take spikes that are in our list
    ids = np.array(ids[spikes_to_take])
    ts = np.array(ts[spikes_to_take]).astype(float) / sampling_frequency

    return ids, ts


def bin_neuron(spike_times, bin_size=.100, window=None):
    """
    Make binned raster for a single neuron
    :param spike_times:
    :param bin_size: in sec
    :param window:
    :return:
    """
    if window is None:
        window = [0, spike_times.max()]
    bins = np.arange(window[0], window[1] + bin_size, bin_size)
    return np.histogram(spike_times, bins)[0]


def bin_neurons(spike_times, neuron_ids, bin_size=None, window=None, plotose=False):
    """
    Make binned raster for many neurons
    :param spike_times:
    :param neuron_ids:
    :param bin_size: in sec
    :param window:
    :param plotose:
    :return:
    """
    if window is None:
        window = [0, spike_times.max()]
    # the following uses an inline for loop (look it up):
    spike_counts = [bin_neuron(spike_times[neuron_ids == neuron_id], bin_size, window)
                    for neuron_id in tqdm(np.unique(neuron_ids))]
    spike_counts = np.vstack(spike_counts)
    raster = xr.DataArray(spike_counts, coords=dict(Time=np.arange(window[0], window[1] + bin_size, bin_size)[:-1],
                                                    Single_unit_id=range(len(np.unique(neuron_ids)))),
                          dims=['Single_unit_id', 'Time'])
    if plotose:
        raster.plot(robust=True)
    return raster


def identify_down_states(ts, bin_size=.05, number_of_neurons_treshold=20, minimum_time_between_states=0.15):
    """
    Find spontaneous periods of quiecence in spiketimes
    :param ts:
    :param bin_size:
    :param number_of_neurons_treshold:
    :param minimum_time_between_states:
    :return:
    """
    lfp = bin_neuron(np.sort(ts), bin_size=bin_size)
    down_states = np.where(lfp < number_of_neurons_treshold)[0]
    down_states_lengths = np.diff(down_states)
    print(f'Eliminating {down_states[1:][down_states_lengths < .15 / bin_size].shape} that are too short')
    down_states = down_states[1:][down_states_lengths > minimum_time_between_states / bin_size]
    print(f'Ended up with {down_states.shape} down states')
    # convert into seconds:
    down_states = down_states * bin_size
    down_states -= .03
    print(down_states[(down_states > 63) & (down_states < 68)])  # compare with raster
    return lfp, down_states


class PSTH:
    """
    A collection of functions dealing with peristimulus time histogram
    """

    @staticmethod
    def make_psth(trial_starts):
        """
        Simple wrapper creating a dataframe with times we want to lock onto
        :param trial_starts: List of times of interest (trials)
        :return: spykes object
        """
        trials = pd.DataFrame()
        trials['trialStart'] = trial_starts
        return trials

    @staticmethod
    def spykes_get_times(s_ts, s_id, debug=False):
        """
        Use spykes library
        :param s_ts:
        :param s_id:
        :param debug:
        :return:
        """

        def print_spyke(spykess):
            [print(len(spykess[i].spiketimes)) for i in range(len(spykess))]

        from spykes.plot import neurovis
        s_id = s_id.astype('int')
        neuron_list = list()
        for iu in np.unique(s_id):
            spike_times = s_ts[s_id == iu]
            if len(spike_times) < 2:
                if debug:
                    print('Too few spiketimes in this unit: ' + str(spike_times))
                else:
                    pass  # neuron_list.append(NeuroVis([],'ram'+str(iu)))
            else:
                neuron = neurovis.NeuroVis(spike_times, name='ram' + str(iu))
                neuron_list.append(neuron)

        if debug:
            print_spyke(neuron_list)
        return neuron_list

    @staticmethod
    def spykes_summary(spikes, spykes_df, event, window=[-100, 100], bin_size=10, fr_thr=.1, plotose=True):
        """

        :param spikes:
        :param spykes_df:
        :param event:
        :param window:
        :param bin_size:
        :param fr_thr:
        :param plotose:
        :return:
        """
        import spykes
        assert window[1] - window[0] > 0, 'Window size must be greater than zero!'
        # filter firing rate
        spikes = [i for i in spikes if i.firingrate > fr_thr]
        pop = spykes.plot.popvis.PopVis(spikes)
        # calculate psth
        mean_psth = pop.get_all_psth(event=event, df=spykes_df, window=window, binsize=bin_size, plot=False)
        assert mean_psth['data'][0].size > 0, 'Empty group PSTH!'
        if plotose:
            # % plot heatmap of average psth
            _ = plt.figure(figsize=(10, 10))
            #        fig.subplots_adjust(hspace=.3)
            # set_trace()
            pop.plot_heat_map(mean_psth, sortby=None, sortorder='ascend', normalize=None,
                              colors=['viridis'])  # or latency

            # %% Population PSTH
            plt.figure()
            pop.plot_population_psth(all_psth=mean_psth, event_name='Event',
                                     colors=([.5, .5, .5], [0, .6, 0]))

        return pop, mean_psth


download_data()

In [ ]:
# Import basic libraries
import matplotlib.pylab as plt
import numpy as np
from tqdm import tqdm

In [ ]:
# @title Figure settings

%config InlineBackend.figure_format = 'retina'

plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

In [ ]:
neuron_ids_huge, spike_times_all_neurons_huge = load_spikes_from_phy('data/')

# Housekeeping detour
## Inspecting variables
## Inspecting functions 

# Visualizing spiketimes without binning 

In [ ]:
# Let's inspect the spikes
print(neuron_ids_huge)
print(spike_times_all_neurons_huge)

# What are the sizes of each of these variables?

In [ ]:
# Limit the number of spikes we're dealing with but keep full variables
neuron_ids = neuron_ids_huge[spike_times_all_neurons_huge < 70]
spike_times_all_neurons = spike_times_all_neurons_huge[spike_times_all_neurons_huge < 70]

In [ ]:
# Let's inspect the timestamps from one neuron
id_of_interest = neuron_ids[300]
spike_times_of_interest = spike_times_all_neurons[neuron_ids == id_of_interest]
print(spike_times_of_interest)

In [ ]:
# Plot spiketimes
plt.plot(spike_times_of_interest);

## What are neurons' identities in this plot? 
## Why is the curve progressively rising?
## What's on the x- and y- axes?


### Exercise 1
Let's inspect the timestamps from one neuron - version 2

In [ ]:
plt.plot(spike_times_of_interest)
plt.title('Timestamps of a neuron')
plt.ylabel('Time')
plt.xlabel('Number of spikes');

 What is a plot like this good for? (Hint: what would the change in firing rate look like?)

### Exercise 2
# Visualizing raster of spiketimes 

**Suggestions**
* Loop variable `step` for 10 steps (`step` takes values from `0` to `9`)
* At each time step
    * Compute the value of `t` with variables `step` and `dt`
    * Compute the value of `i`
    * Print `i`

In [ ]:
## TODO for students: compute the mean squared error
# Uncomment below to get started


# Raster of one neuron, by hand
# plt.xlim([0, spike_times_of_interest.max()])
# for timestamp in spike_times_of_interest:
#     plt.scatter(x=..., y=..., marker='|', color='black')
#     plt.show()
#     plt.pause(.01)

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/tutorials/solutions/spikes_tutorial_Solution_ca49f968.py)

*Example output:*

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_0.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_1.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_2.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_3.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_4.png>

<img alt='Solution hint' align='left' width=556 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_5.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_6.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_7.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_8.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_9.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_10.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_11.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_12.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_13.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_14.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_15.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_16.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_17.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_18.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_19.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_20.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_21.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_22.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_23.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_24.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_25.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_26.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_27.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_28.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_29.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_30.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_31.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_32.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_33.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_34.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_35.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_36.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_37.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_38.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_39.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_40.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_41.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_42.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_43.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_44.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_45.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_46.png>

<img alt='Solution hint' align='left' width=556 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_47.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_48.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_49.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_50.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_51.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_52.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_53.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_54.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_55.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_56.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_57.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_58.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_59.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_60.png>

<img alt='Solution hint' align='left' width=556 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_61.png>

<img alt='Solution hint' align='left' width=556 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_62.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_63.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_64.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_65.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_66.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_67.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_68.png>

<img alt='Solution hint' align='left' width=556 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_69.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_70.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_71.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_72.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_73.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_74.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_75.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_76.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_77.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_78.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_79.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_80.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_81.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_82.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_83.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_84.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_85.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_86.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_87.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_88.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_89.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_90.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_91.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_92.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_93.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_94.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_95.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_96.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_97.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_98.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_99.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_100.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_101.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_102.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_103.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_104.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_105.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_106.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_107.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_108.png>

<img alt='Solution hint' align='left' width=556 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_109.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_110.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_111.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_112.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_113.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_114.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_115.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_116.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_117.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_118.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_119.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_120.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_121.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_122.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_123.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_124.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_125.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_126.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_127.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_128.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_129.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_130.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_131.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_132.png>

<img alt='Solution hint' align='left' width=556 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_133.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_134.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_135.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_136.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_137.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_138.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_139.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_140.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_141.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_142.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_143.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_144.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_145.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_146.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_147.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_148.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_149.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_150.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_151.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_152.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_153.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_154.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_155.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_156.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_157.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_158.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_159.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_160.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_161.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_162.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_163.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_164.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_165.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_166.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_167.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_168.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_169.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_170.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_171.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_172.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_173.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_174.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_175.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_176.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_177.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_178.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_179.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_180.png>

<img alt='Solution hint' align='left' width=556 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_181.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_182.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_183.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_184.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_185.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_186.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_187.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_188.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_189.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_190.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_191.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_192.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_193.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_194.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_195.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_196.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_197.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_198.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_199.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_200.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_201.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_202.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_203.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_204.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_205.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_206.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_207.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_208.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_209.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_210.png>

<img alt='Solution hint' align='left' width=556 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_211.png>

<img alt='Solution hint' align='left' width=556 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_212.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_213.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_214.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_215.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_216.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_217.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_218.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_219.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_220.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_221.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_222.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_223.png>

<img alt='Solution hint' align='left' width=556 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_224.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_225.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_226.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_227.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_228.png>

<img alt='Solution hint' align='left' width=556 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_229.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_230.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_231.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_232.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_233.png>

<img alt='Solution hint' align='left' width=556 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_234.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_235.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_236.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_237.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_238.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_239.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_240.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_241.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_242.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_243.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_244.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_245.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_246.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_247.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_248.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_249.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_250.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_251.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_252.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_253.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_254.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_255.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_256.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_257.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_258.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_259.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_260.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_261.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_262.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_263.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_264.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_265.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_266.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_267.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_268.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_269.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_270.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_271.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_272.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_273.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_274.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_275.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_276.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_277.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_278.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_279.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_280.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_281.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_282.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_283.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_284.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_285.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_286.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_287.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_288.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_289.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_290.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_291.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_292.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_293.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_294.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_295.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_296.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_297.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_298.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_299.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_300.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_301.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_302.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_303.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_304.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_305.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_306.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_307.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_308.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_309.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_310.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_311.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_312.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_313.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_314.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_315.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_316.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_317.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_318.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_319.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_320.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_321.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_322.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_323.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_324.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_325.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_326.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_327.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_328.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_329.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_330.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_331.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_332.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_333.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_334.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_335.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_336.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_337.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_338.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_339.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_340.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_341.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_342.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_343.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_344.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_345.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_346.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_347.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_348.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_349.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_350.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_351.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_352.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_353.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_354.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_355.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_356.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_357.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_358.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_359.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_360.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_361.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_362.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_363.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_364.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_365.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_366.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_367.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_368.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_369.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_370.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_371.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_372.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_373.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_374.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_375.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_376.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_377.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_378.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_379.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_380.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_381.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_382.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_383.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_384.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_385.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_386.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_387.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_388.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_389.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_390.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_391.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_392.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_393.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_394.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_395.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_396.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_397.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_398.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_399.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_400.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_401.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_402.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_403.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_404.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_405.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_406.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_407.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_408.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_409.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_410.png>

<img alt='Solution hint' align='left' width=557 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_411.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_412.png>

<img alt='Solution hint' align='left' width=558 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/tutorials/static/spikes_tutorial_Solution_ca49f968_413.png>



This is our first For loop! How does it work?


In [ ]:
# Whole raster for one neuron, no loop
# %matplotlib tk # slide housekeeping, nothing to see here
plt.scatter(x=spike_times_of_interest,
            y=np.ones(spike_times_of_interest.shape[0]),
            marker='|');

In [ ]:
# For all neurons now
plt.scatter(x=spike_times_all_neurons,
            y=neuron_ids,
            marker='|', c='k', alpha=.7);

# visualizing binned raster 


In [ ]:
# Use my function with xarray
raster = bin_neurons(spike_times_all_neurons, neuron_ids, bin_size=.05)
print(raster)  # xarray

In [ ]:
raster.plot(robust=True);
# plt.show()

In [ ]:
# Convenient one-liners to "slice" the raster by time or neurons
raster.sel(Single_unit_id=slice(20, 1200), Time=slice(60, 65)).plot(robust=True);
## Do you see the difference? We are missing-ish the up states

In [ ]:
# PSTH
lfp, down_states = identify_down_states(spike_times_all_neurons_huge)

In [ ]:
# PSTH based on homemade code
raster = bin_neurons(spike_times_all_neurons_huge, neuron_ids_huge, bin_size=.005)
psth = []
for down_state in tqdm(down_states):
    # get raster
    raster_now = raster.sel(Time=slice(down_state - .100,
                                       down_state + .300))
    # add raster to psth
    psth.append(raster_now.sel(Single_unit_id=241).values[:80])  # Note no equals sign
psth_all = np.vstack(psth)
# With for loop as the basic building block, you can do anything!
## plot psth
plt.pcolormesh(psth_all);
## Q: What are the axes?

In [ ]:
# ============= Kording psth toolbox is a lot easier ===============
# make a psth object using the toolbox:
psth_object = PSTH.make_psth(down_states)
# make raster using toolbox:
spykes_times = PSTH.spykes_get_times(spike_times_all_neurons_huge, neuron_ids_huge)
# Get mean PSTH for all neurons
pop, all_psth = PSTH.spykes_summary(spikes=spykes_times, spykes_df=psth_object,
                                    event='trialStart', window=[-300, 400], bin_size=5, plotose=True)
# Plot mean PSTH for all neurons
pop.plot_heat_map(all_psth, sortby='rate', sortorder='ascend', normalize=None, colors=['viridis']);  # or latency
## Note the periodicity
## Q: Do you think if you flip the detection, the result will be similar and opposite? Hint: Inspect the raster.
# Test your hypothesis by setting down_states = np.where(lfp > 90)[0]